In [1]:
import os
os.chdir('..')
print(os.getcwd())

/workspace/rendering


In [6]:

from utils.mano import ManoLayer
from utils.utils import load_ho_meta, get_obj_mesh, apply_transform_to_mesh, add_trimesh_to_fig
import trimesh
import plotly.graph_objects as go
import trimesh
import numpy as np
import torch
from scipy.spatial.transform import Rotation as R

# load object
obj_mesh = trimesh.load_mesh('data/models/006_mustard_bottle/textured_simple.obj')


class AmbientSensor():
    def __init__(self, cell_length=0.018):
        # 3次元格子の生成
        x = np.linspace(0, -cell_length*10, 10) 
        y = np.linspace(0, -cell_length*10, 10)
        z = np.linspace(0, cell_length*10, 10)
        X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
        self.lattice = torch.from_numpy(np.column_stack((X.ravel(), Y.ravel(), Z.ravel()))).to(torch.float32)
        trans = torch.tensor([[0.04,0.0,-0.09]])
        self.lattice += trans
        self.lattice_origin = self.lattice.clone()
        
        self.values = torch.rand(self.lattice.shape[0])
        self.cell_length = cell_length


    # set transform to mano hand
    def calc_ambient(self, mano_hand, obj_mesh):
        middle_pos = mano_hand.joints[:, 4]#[np.newaxis, :]
        rotvec = mano_hand.global_orient[0]
        rot = torch.from_numpy(R.from_rotvec(rotvec).as_matrix()).to(torch.float32)
        
        self.lattice = self.lattice_origin @ rot.T + middle_pos
        
        # get signed distance from object mesh to lattice
        sd = torch.from_numpy(obj_mesh.nearest.signed_distance(self.lattice)).to(torch.float32)
        
        # calc ambient value
        self.values[sd < (-self.cell_length)] = 0
        self.values[sd > (-self.cell_length)] = 1 - sd[sd > (-self.cell_length)] / self.cell_length
        self.values[sd > 0] = 1
        return self.values


    def get_spheres(self):
        # 3次元格子を描画
        lattice_spheres = [trimesh.primitives.Sphere(radius=0.005, center=point) for point in self.lattice]



mano = ManoLayer()
anno = load_ho_meta()

mano_hand = mano(anno)

obj_verts, obj_faces, obj_uv, obj_map = get_obj_mesh(anno)
obj_verts = apply_transform_to_mesh(obj_verts, anno)


# print(mano_hand.shape, mano.faces.shape)
mano_mesh = trimesh.Trimesh(vertices=mano_hand.vertices[0], faces=mano.faces, )
obj_mesh = trimesh.Trimesh(vertices=obj_verts, faces=obj_faces)

amb = AmbientSensor()
amb.calc_ambient(mano_hand, obj_mesh)
# print(amb.values)
print(amb.values.min(), amb.values.max())

fig = go.Figure()
add_trimesh_to_fig(fig, mano_mesh, opacity=0.8)
add_trimesh_to_fig(fig, obj_mesh, opacity=0.8)

fig.add_trace(
    go.Scatter3d(
        x=mano_hand.joints[:, 4, 0],
        y=mano_hand.joints[:, 4, 1],
        z=mano_hand.joints[:, 4, 2],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=1
        )
    )
)

fig.add_trace(
    go.Scatter3d(
        x=amb.lattice[:, 0],
        y=amb.lattice[:, 1],
        z=amb.lattice[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=amb.values,
            colorscale='Viridis',
            opacity=0.5
        )
    )
)

torch.Size([8251, 3]) torch.Size([3, 3]) torch.Size([3])
tensor(0.) tensor(1.9967)


In [9]:
# input info
print(amb.values)
print(amb.values.shape)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [3]:
mano_hand.betas.shape

torch.Size([1, 10])

In [4]:
import numpy as np


root_pose = torch.zeros(1, 3)
hand_pose = torch.zeros(1, 45)
betas = torch.zeros(1, 10)
trans = torch.zeros(1, 3)
output = mano.mano_layer['right'](global_orient=root_pose, hand_pose=hand_pose, betas=betas, transl=trans)
mano_mesh = trimesh.Trimesh(vertices=output.vertices[0], faces=mano.faces, )

amb = AmbientSensor()


fig = go.Figure()
add_trimesh_to_fig(fig, mano_mesh, opacity=0.5)

u, v, w = np.array([[1,0,0], [0,1,0], [0,0,1]]).T

# fig.add_trace(
# go.Cone(
#         x=np.zeros([3, 3]), 
#         y=np.zeros([3, 3]),
#         z=np.zeros([3, 3]),
#         u=u, v=v, w=w,
#         sizemode="scaled",
#         sizeref=10,
#     )
# )
fig.add_trace(
    go.Scatter3d(
        x=[0, 0.1], y=[0, 0], z=[0, 0],
        mode='lines', name='x', line=dict(color='red')
    ))
fig.add_trace(
    go.Scatter3d(
        x=[0, 0], y=[0, 0.1], z=[0, 0],
        mode='lines', name='y', line=dict(color='green')
    ))
fig.add_trace(
    go.Scatter3d(
        x=[0, 0], y=[0, 0], z=[0, 0.1],
        mode='lines', name='z', line=dict(color='blue')
    ))

x, y, z = amb.lattice.T
fig.add_trace(
    go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(size=2, colorscale='Viridis', opacity=0.5)
    )
)